In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from os.path import join

from jupyter_utils.progress_bar import ProgressBar

from nabirds import CUB_Annotations, NAB_Annotations
from nabirds.dataset import RevealedPartMixin, CroppedPartMixin, AnnotationsReadMixin

from chainer_addons.models import ResnetLayers, InceptionV3, VGG19Layers
from chainer_addons.dataset import PreprocessMixin

from chainer import cuda
from chainer.iterators import MultithreadIterator
from chainer.dataset.convert import concat_examples

import chainer
chainer.__version__

'4.2.0'

In [22]:
BASE_DIR = "/home/korsch/Data"

ARCH="resnet"
DATA="cub200_11"

DATA_DIR = join(BASE_DIR, "DATASETS", "birds", DATA)
WEIGHTS = join(BASE_DIR, "MODELS", ARCH, "ft_cub200", "model.npz")


DATASETS = dict(
    cub200_11=CUB_Annotations,
    nabirds=NAB_Annotations,
)

LABEL_SHIFT = 1

MODELS = dict(
    resnet=ResnetLayers,
    inception=InceptionV3,
    vgg19=VGG19Layers,
)

model_cls = MODELS.get(ARCH)
annot_cls = DATASETS.get(DATA)

GPU = 1

RATIO = 1 / 5
SIZE = None


In [4]:
from abc import ABC

import matplotlib.pyplot as plt

class _mixin(ABC):
    def get_example(self, i):
        im_obj = super(_mixin, self).get_example(i)
        im, parts, lab = im_obj.as_tuple()
        root = im_obj
        while root.parent is not None:
            root = root.parent
        self.original = self.preprocess(root.im)
        return im, lab
    
class Dataset(
    PreprocessMixin, _mixin,
    CroppedPartMixin, AnnotationsReadMixin):
    
    def get_example(self, i):
        im, lab = super(Dataset, self).get_example(i)
        im = np.vstack([im, self.original[None]])
        return im, lab
        
    def __init__(self, split, annot, *args, **kw):
        global model_cls
        super(Dataset, self).__init__(
            annotations=annot,
            uuids=annot.uuids[split],
            
            size=SIZE or model_cls.meta.input_size,
            preprocess=model_cls.prepare,
            
            crop_to_bb=False,
            crop_uniform=True,
            
            uniform_parts=True,
            ratio=RATIO,
            parts_in_bb=False,
            
            return_part_crops=True,
            *args, **kw
        )

In [13]:
%%time
print("Loading annotations")
annot = annot_cls(DATA_DIR)

print("Creating train and test datasets")
train, test = [Dataset(s, annot) for s in [annot.train_split, annot.test_split]]

n_classes = LABEL_SHIFT + len(np.unique(annot.labels))

print("Loaded {} train and {} test samples with {} classes".format(len(train), len(test), n_classes))

Loading annotations
Creating train and test datasets
Loaded 5994 train and 5794 test samples with 201 classes
CPU times: user 708 ms, sys: 46.7 ms, total: 755 ms
Wall time: 755 ms


In [14]:
model = model_cls()
model.load_for_inference(weights=WEIGHTS, n_classes=n_classes)

if GPU >= 0:
    cuda.get_device(GPU).use()
    model.to_gpu(GPU)

In [15]:
def extract(data, n_jobs=None):
    bar = ProgressBar()
    feats = None
    n_samples = len(data)
    it = MultithreadIterator(data, batch_size=1, repeat=False, shuffle=False, n_threads=n_jobs)
    for i, batch in enumerate(bar(it, every=1, size=n_samples)):
        X, y = concat_examples(batch, device=GPU)
        # number of parts is our batch size!
        X = X[0]
        if feats is None:
            n_parts = X.shape[0]
            feat_size = model_cls.meta.feature_size
            feats = np.zeros((n_samples, n_parts, feat_size), dtype=np.float32)
        
        part_feats = model(X, layer_name=model_cls.meta.feature_layer)
        part_feats.to_cpu()
        
        feats[i] = part_feats.array
    
    return feats


with chainer.using_config("train", False), chainer.no_backprop_mode():
    train_feats = extract(train)
    test_feats = extract(test)

In [23]:
n_parts = int(1 / RATIO)**2 + 1
fmt = "{}_{}parts_uniform_{}.npz"
LOAD = True

In [ ]:
def dataset_params(data):
    return dict(
        crop_to_bb=data.crop_to_bb,
        crop_uniform=data.crop_uniform,
        uniform_parts=data.uniform_parts,
        size=data.size,
        ratio=data.ratio,
    )

In [24]:
%%time 
if LOAD:
    train_feats = np.load(fmt.format("train", n_parts, ARCH))["features"]
    test_feats = np.load(fmt.format("val", n_parts, ARCH))["features"]
else:
    np.savez(join("output", fmt.format("train", n_parts, ARCH)), 
             features=train_feats, 
             **dataset_params(train))
    np.savez(join("output", fmt.format("val", n_parts, ARCH)), 
             features=test_feats, 
             **dataset_params(test))

CPU times: user 5.73 s, sys: 1.9 s, total: 7.63 s
Wall time: 7.6 s


In [30]:
test_feats.shape

(5794, 26, 4096)

# Train a baseline SVM

In [19]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
y, y_test = annot.labels[annot.train_split], annot.labels[annot.test_split]
%time clf.fit(train_feats.reshape(len(train_feats), -1), y)

CPU times: user 16min 8s, sys: 1.9 s, total: 16min 10s
Wall time: 16min 3s


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [20]:
%time clf.score(train_feats.reshape(len(train_feats), -1), y)

CPU times: user 17.4 s, sys: 5.27 s, total: 22.7 s
Wall time: 1.69 s


1.0

In [21]:
%time clf.score(test_feats.reshape(len(test_feats), -1), y_test)

CPU times: user 16.5 s, sys: 4.86 s, total: 21.3 s
Wall time: 1.57 s


0.7340352088367277

In [25]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
y, y_test = annot.labels[annot.train_split], annot.labels[annot.test_split]
%time clf.fit(train_feats.reshape(len(train_feats), -1), y)

CPU times: user 18min 55s, sys: 2.93 s, total: 18min 57s
Wall time: 18min 49s


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [26]:
%time clf.score(train_feats.reshape(len(train_feats), -1), y)

CPU times: user 33.7 s, sys: 7.82 s, total: 41.5 s
Wall time: 3.22 s


1.0

In [27]:
%time clf.score(test_feats.reshape(len(test_feats), -1), y_test)

CPU times: user 32.7 s, sys: 7.89 s, total: 40.5 s
Wall time: 3.12 s


0.6839834311356576

In [29]:
test_feats.shape

(5794, 26, 4096)